In [ ]:
#conect with google disk
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import numpy as np
import os, sys
import glob

import json

In [ ]:
 %%time
path = '/content/drive/MyDrive/Colab Notebooks/DS_test/json/'

#for file in (glob.glob(os.path.join(path, "*.json"))):
#    data += [json.loads(line) for line in open(file, 'r')]

with open('/content/drive/MyDrive/Colab Notebooks/DS_test/json/part-00001-900ddb8f-b0f1-4a8f-9dba-8ff2cdd88b38-c000.json') as f:
    data = [json.loads(line) for line in f]


CPU times: user 5.89 s, sys: 1.64 s, total: 7.53 s
Wall time: 10.1 s


In [ ]:
%%time
df = pd.DataFrame()
for file in (glob.glob(os.path.join(path, "*.json"))):
    with open(file) as f:
        data = [json.loads(line) for line in f]
        df1 = pd.json_normalize(data)
    df = pd.concat([df1, df], ignore_index=True)

df

CPU times: user 8min 27s, sys: 16.9 s, total: 8min 44s
Wall time: 9min


In [ ]:
df.shape

(3473877, 27)

### №3 What is(are) the main time period(s) in the data?
##### datetime
We have 2 time period:
2019/03/22 - 2019/03/29
2020/03/20 - 2020/03/27

In [ ]:
%%time
import datetime
df['epochMillis_new'] = (df['epochMillis'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S.%f')))
df['date'] = pd.to_datetime(df['epochMillis_new'].apply(lambda x: x.split(' ')[0]))
df['time'] = df['epochMillis_new'].apply(lambda x: (x.split(' ')[1].split('.')[0]))


CPU times: user 20.7 s, sys: 10.6 ms, total: 20.7 s
Wall time: 20.8 s


In [ ]:
df.head(5)

,epochMillis,mmsi,olson_timezone,imo,callSign,destination,position.latitude,position.longitude,navigation.navCode,navigation.navDesc,navigation.courseOverGround,navigation.heading,navigation.rateOfTurn,navigation.speedOverGround,vesselDetails.name,vesselDetails.typeName,vesselDetails.typeCode,vesselDetails.draught,vesselDetails.length,vesselDetails.width,vesselDetails.flagCode,vesselDetails.flagCountry,port.unlocode,port.name,port.latitude,port.longitude,cargoDetails,epochMillis_new,date,time
0,1553847836000,413372730,Asia/Shanghai,0.0,FYUA$,SHANGHAI,31.334450,121.550385,16,Unknown,317.3,0.0,0.0,0.0,TONGYUAN10,Tanker,80,0.3,42.0,9.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-29 08:23:56.000000,2019-03-29,08:23:56
1,1553848022000,412018031,Asia/Shanghai,NaN,00371,NaN,30.844887,122.295370,16,Unknown,128.7,480.0,0.0,8.2,ZHEPINHUYU00371,Fishing,30,NaN,28.0,4.0,412,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-29 08:27:02.000000,2019-03-29,08:27:02
2,1553848135000,413444250,Asia/Shanghai,0.0,BLGS8,DONG YING,30.723333,122.274870,0,Under Way Using Engine,26.5,28.0,0.0,11.3,XING LONG ZHOU 172,Tanker,80,4.5,119.0,18.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-29 08:28:55.000000,2019-03-29,08:28:55
3,1553848288000,412378760,Asia/Shanghai,2176.0,BIAL2,YUEQINWAN,31.333086,121.557630,16,Unknown,0.0,0.0,0.0,0.0,ZHONGYUANRANGONG 01,Tanker,83,9.2,53.0,9.0,412,China,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2019-03-29 08:31:28.000000,2019-03-29,08:31:28
4,1553848430000,412365857,Asia/Shanghai,NaN,NaN,NaN,32.118640,121.499466,16,Unknown,319.0,0.0,0.0,0.2,SUHAIMENYU16271,Unknown,100,NaN,NaN,NaN,412,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-29 08:33:50.000000,2019-03-29,08:33:50


In [ ]:
df['epochMillis_new'] = pd.to_datetime(df['epochMillis_new'])

In [ ]:
%%time
#df['time'] = pd.to_datetime(df['time']).apply(lambda x: x.strftime('%H:%M:%S.%f'))

CPU times: user 3min, sys: 397 ms, total: 3min 1s
Wall time: 3min 1s


In [ ]:
df.dtypes

epochMillis                             int64
mmsi                                    int64
olson_timezone                         object
imo                                   float64
callSign                               object
destination                            object
position.latitude                     float64
position.longitude                    float64
navigation.navCode                      int64
navigation.navDesc                     object
navigation.courseOverGround           float64
navigation.heading                    float64
navigation.rateOfTurn                 float64
navigation.speedOverGround            float64
vesselDetails.name                     object
vesselDetails.typeName                 object
vesselDetails.typeCode                  int64
vesselDetails.draught                 float64
vesselDetails.length                  float64
vesselDetails.width                   float64
vesselDetails.flagCode                  int64
vesselDetails.flagCountry         

In [ ]:
df

,epochMillis,mmsi,olson_timezone,imo,callSign,destination,position.latitude,position.longitude,navigation.navCode,navigation.navDesc,navigation.courseOverGround,navigation.heading,navigation.rateOfTurn,navigation.speedOverGround,vesselDetails.name,vesselDetails.typeName,vesselDetails.typeCode,vesselDetails.draught,vesselDetails.length,vesselDetails.width,vesselDetails.flagCode,vesselDetails.flagCountry,port.unlocode,port.name,port.latitude,port.longitude,cargoDetails,epochMillis_new,date,time
0,1553847836000,413372730,Asia/Shanghai,0.0,FYUA$,SHANGHAI,31.334450,121.550385,16,Unknown,317.3,0.0,0.0,0.0,TONGYUAN10,Tanker,80,0.3,42.0,9.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-29 08:23:56,2019-03-29,08:23:56.000000
1,1553848022000,412018031,Asia/Shanghai,NaN,00371,NaN,30.844887,122.295370,16,Unknown,128.7,480.0,0.0,8.2,ZHEPINHUYU00371,Fishing,30,NaN,28.0,4.0,412,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-29 08:27:02,2019-03-29,08:27:02.000000
2,1553848135000,413444250,Asia/Shanghai,0.0,BLGS8,DONG YING,30.723333,122.274870,0,Under Way Using Engine,26.5,28.0,0.0,11.3,XING LONG ZHOU 172,Tanker,80,4.5,119.0,18.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-29 08:28:55,2019-03-29,08:28:55.000000
3,1553848288000,412378760,Asia/Shanghai,2176.0,BIAL2,YUEQINWAN,31.333086,121.557630,16,Unknown,0.0,0.0,0.0,0.0,ZHONGYUANRANGONG 01,Tanker,83,9.2,53.0,9.0,412,China,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2019-03-29 08:31:28,2019-03-29,08:31:28.000000
4,1553848430000,412365857,Asia/Shanghai,NaN,NaN,NaN,32.118640,121.499466,16,Unknown,319.0,0.0,0.0,0.2,SUHAIMENYU16271,Unknown,100,NaN,NaN,NaN,412,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-29 08:33:50,2019-03-29,08:33:50.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3473872,1553471408000,413764058,Asia/Shanghai,0.0,NaN,NaN,30.947575,121.216095,16,Unknown,2.0,0.0,0.0,7.8,HU HUAN HUO 30,Other,90,0.0,17.0,5.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-24 23:50:08,2019-03-24,23:50:08.000000
3473873,1553471514000,566400000,Asia/Shanghai,9736638.0,9V3031,SHANGAI,31.090883,121.463104,0,Under Way Using Engine,102.4,158.0,0.0,0.1,MAREX NOA,Tanker,84,7.0,124.0,20.0,566,Singapore,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2019-03-24 23:51:54,2019-03-24,23:51:54.000000
3473874,1553471653000,412760770,Asia/Shanghai,0.0,BLGL7,GUANG ZHOU,30.746483,122.932560,0,Under Way Using Engine,181.5,183.0,0.0,10.0,HONG PU 96,Cargo,79,8.0,140.0,20.0,412,China,CNSHG,SHANGHAI PT,31.334994,121.659069,No Additional Information,2019-03-24 23:54:13,2019-03-24,23:54:13.000000
3473875,1553471750000,413373640,Asia/Shanghai,0.0,NaN,SHANGHAI,31.391459,121.662190,5,Moored,26.6,0.0,0.0,0.0,ZHEN HUA TUO 12,Tug,52,3.5,0.0,0.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-24 23:55:50,2019-03-24,23:55:50.000000


In [ ]:
data_time = pd.DataFrame({'date': df['date'].value_counts().argsort(axis='index').index, 
                          'col': df['date'].value_counts().values}, 
                         index = df['date'].value_counts().argsort(axis='index') )
data_time['date'].sort_values(ascending=False)

date
8    2020-03-27
10   2020-03-26
5    2020-03-25
2    2020-03-24
4    2020-03-23
7    2020-03-22
11   2020-03-21
0    2020-03-20
15   2019-03-29
14   2019-03-28
13   2019-03-27
12   2019-03-26
9    2019-03-25
3    2019-03-24
6    2019-03-23
1    2019-03-22
Name: date, dtype: datetime64[ns]

# Answer:

#### We have 2 time period: 
<li>  2019/03/22 - 2019/03/29 </li>
<li>  2020/03/20 - 2020/03/27</li>

# №4 Which are the top three most sparse variables?

In [ ]:
df.fillna(0, inplace=True)
pd.DataFrame({'col': df.columns,
              'count': [(df[i]==0).sum()+((df[i]=='0')).sum() for i in df.columns]}).sort_values(by=['count']).tail(3)

,col,count
3,imo,2646208
26,cargoDetails,3140605
12,navigation.rateOfTurn,3361992


# Answer:

The top three most sparse variables are navigation.rateOfTurn	(3361992) cargoDetails (3140605), imo (2646208).

# №5 What region(s) of the world and ocean port(s) does this data represent? Provide evidence to justify your answer.

In [ ]:
print('The region(s) of the world:', df['olson_timezone'].value_counts())
print('The ocean port:', df['port.name'].value_counts())

The region(s) of the world: Asia/Shanghai    3473877
Name: olson_timezone, dtype: int64
The ocean port: SHANGHAI PT    3473877
Name: port.name, dtype: int64


# Answer:
This data representThe the Asia/Shanghai region of the worl,
the SHANGHAI PT ocean port.

# №6 Provide a frequency tabulation of the various Navigation Codes & Descriptions (i.e., navCode & NavDesc). Optionally, provide any additional statistics you find interesting.

# Answer:

In [ ]:
pd.crosstab(df['navigation.navCode'], df['navigation.navDesc'])

navigation.navDesc,Aground,At Anchor,Constrained By Her Draught,Engaged In Fishing,Moored,Not Defined,Not Under Command,Reserved For Future Amendment,Reserved For Future Use,Restricted Manoeuvrability,Under Way Using Engine,Underway Sailing,Unknown
navigation.navCode,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,1063676,0,0
1,0,426433,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,3471,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,8237,0,0,0
4,0,0,1483,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,554133,0,0,0,0,0,0,0,0
6,401,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,321,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,24889,0


In [ ]:
print("df['navigation.navDesc']& df['navigation.navCode'].mean():",
  pd.crosstab( df['navigation.navDesc'], df['navigation.navCode']).mean())
print("df['navigation.navDesc']& df['navigation.navCode'].sum():",
  pd.crosstab( df['navigation.navDesc'], df['navigation.navCode']).sum())

df['navigation.navDesc']& df['navigation.navCode'].mean(): navigation.navCode
0      81821.230769
1      32802.538462
2        267.000000
3        633.615385
4        114.076923
5      42625.615385
6         30.846154
7         24.692308
8       1914.538462
9        104.538462
11        98.230769
12         0.538462
13        67.307692
15      2256.153846
16    104460.384615
dtype: float64
df['navigation.navDesc']& df['navigation.navCode'].sum(): navigation.navCode
0     1063676
1      426433
2        3471
3        8237
4        1483
5      554133
6         401
7         321
8       24889
9        1359
11       1277
12          7
13        875
15      29330
16    1357985
dtype: int64


In [ ]:
#additional interesting statistics :
df.pivot_table(['navigation.navCode'], ['navigation.navDesc'])
'''or'''
print("df['navigation.navCode']& df['navigation.navDesc'].max:", 
      df.pivot_table(['navigation.navCode'], ['navigation.navDesc'], aggfunc=(np.max)).sort_values(['navigation.navCode']))

df['navigation.navCode']& df['navigation.navDesc'].max:                                navigation.navCode
navigation.navDesc                               
Under Way Using Engine                          0
At Anchor                                       1
Not Under Command                               2
Restricted Manoeuvrability                      3
Constrained By Her Draught                      4
Moored                                          5
Aground                                         6
Engaged In Fishing                              7
Underway Sailing                                8
Reserved For Future Amendment                   9
Reserved For Future Use                        13
Not Defined                                    15
Unknown                                        16


In [ ]:
navigation = (pd.crosstab(df['navigation.navCode'], df['navigation.navDesc']).sum()).sort_values( ascending=False)
navigation 

navigation.navDesc
Unknown                          1357985
Under Way Using Engine           1063676
Moored                            554133
At Anchor                         426433
Not Defined                        29330
Underway Sailing                   24889
Restricted Manoeuvrability          8237
Not Under Command                   3471
Reserved For Future Use             2159
Constrained By Her Draught          1483
Reserved For Future Amendment       1359
Aground                              401
Engaged In Fishing                   321
dtype: int64

In [ ]:
pd.crosstab(df['navigation.navCode'], df['navigation.navDesc']).sum()
#df.pivot_table(['navigation.navCode'], ['navigation.navDesc'])

navigation.navDesc
Aground                              401
At Anchor                         426433
Constrained By Her Draught          1483
Engaged In Fishing                   321
Moored                            554133
Not Defined                        29330
Not Under Command                   3471
Reserved For Future Amendment       1359
Reserved For Future Use             2159
Restricted Manoeuvrability          8237
Under Way Using Engine           1063676
Underway Sailing                   24889
Unknown                          1357985
dtype: int64

#  №7 For MMSI = 205792000, provide the following report:
<ol><li> Limit the data to only the TOP 5 Navigation Codes based from the response to question 6 </li>
<li>Provide the final state for each series of contiguous events with the same Navigation Code; series may be interrupted by other </li>
<li>series, but each contiguous series must be its own record.</li> <ol>
Final report should include at least the following fields/columns:
<li>mmsi = the MMSI of the vessel 
timestamp = the timestamp of the last event in that contiguous series Navigation </li>
<li>Code = the navigation code (i.e., navigation.navCode) Navigation Description = the navigation code description (i.e., navigation.navDesc) </li>
<li>lead time (in Milliseconds) = the time difference in milliseconds between the last and first timestamp of that particular series of the same contiguous navigation codes</li>



In [ ]:
#df.sort_values(by=['hours-per-week', 'salary'], ascending=[False, True])

In [ ]:
mmsi_205792000 = df.loc[ (df['mmsi'] == 205792000) &
       ((df['navigation.navDesc'] == navigation.index[0]) | 
       ( df['navigation.navDesc'] == navigation.index[1]) |  
       ( df['navigation.navDesc'] == navigation.index[2]) | 
       ( df['navigation.navDesc'] == navigation.index[3]) |  
       ( df['navigation.navDesc'] == navigation.index[4])), :].copy()

#df.loc[(df['mmsi'] == 205792000) , : ]['navigation.navDesc'].value_counts() # all  ['navigation.navDesc'] in top-5
mmsi_205792000

,epochMillis,mmsi,olson_timezone,imo,callSign,destination,position.latitude,position.longitude,navigation.navCode,navigation.navDesc,navigation.courseOverGround,navigation.heading,navigation.rateOfTurn,navigation.speedOverGround,vesselDetails.name,vesselDetails.typeName,vesselDetails.typeCode,vesselDetails.draught,vesselDetails.length,vesselDetails.width,vesselDetails.flagCode,vesselDetails.flagCountry,port.unlocode,port.name,port.latitude,port.longitude,cargoDetails,epochMillis_new,date,time
56121,1584994411000,205792000,Asia/Shanghai,9400198.0,ONKM,CNSHA,31.264668,122.670334,1,At Anchor,232.3,322.0,1.116007,0.1,TOUCAN HUNTER,Cargo,71,10.3,269.0,33.0,205,Belgium,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2020-03-23 20:13:31,2020-03-23,20:13:31.000000
61738,1585056380000,205792000,Asia/Shanghai,9400198.0,ONKM,CNSHA,31.377527,121.573906,5,Moored,115.9,116.0,0.000000,0.0,TOUCAN HUNTER,Cargo,71,10.4,269.0,33.0,205,Belgium,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2020-03-24 13:26:20,2020-03-24,13:26:20.000000
61748,1585057820000,205792000,Asia/Shanghai,9400198.0,ONKM,CNSHA,31.377500,121.573830,5,Moored,115.8,116.0,0.000000,0.0,TOUCAN HUNTER,Cargo,71,10.4,269.0,33.0,205,Belgium,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2020-03-24 13:50:20,2020-03-24,13:50:20.000000
68059,1584962010000,205792000,Asia/Shanghai,9400198.0,ONKM,CNSHA,31.264500,122.665665,1,At Anchor,77.0,77.0,0.000000,0.0,TOUCAN HUNTER,Cargo,71,10.3,269.0,33.0,205,Belgium,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2020-03-23 11:13:30,2020-03-23,11:13:30.000000
83560,1585100300000,205792000,Asia/Shanghai,9400198.0,ONKM,CNSHA,31.377500,121.574000,5,Moored,115.8,116.0,0.000000,0.0,TOUCAN HUNTER,Cargo,71,10.4,269.0,33.0,205,Belgium,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2020-03-25 01:38:20,2020-03-25,01:38:20.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3366678,1584985051000,205792000,Asia/Shanghai,9400198.0,ONKM,CNSHA,31.268833,122.669334,1,At Anchor,207.6,208.0,0.000000,0.0,TOUCAN HUNTER,Cargo,71,10.3,269.0,33.0,205,Belgium,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2020-03-23 17:37:31,2020-03-23,17:37:31.000000
3372967,1584969211000,205792000,Asia/Shanghai,9400198.0,ONKM,CNSHA,31.269167,122.666664,1,At Anchor,133.5,134.0,5.401475,0.1,TOUCAN HUNTER,Cargo,71,10.3,269.0,33.0,205,Belgium,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2020-03-23 13:13:31,2020-03-23,13:13:31.000000
3399332,1585018990000,205792000,Asia/Shanghai,9400198.0,ONKM,CNSHA,31.250500,121.976830,0,Under Way Using Engine,280.7,281.0,5.401475,10.9,TOUCAN HUNTER,Cargo,71,10.4,269.0,33.0,205,Belgium,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2020-03-24 03:03:10,2020-03-24,03:03:10.000000
3424055,1585124137000,205792000,Asia/Shanghai,9400198.0,ONKM,CNYTN,31.101791,122.351150,0,Under Way Using Engine,93.5,91.0,-2.187374,11.6,TOUCAN HUNTER,Cargo,71,11.7,269.0,33.0,205,Belgium,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2020-03-25 08:15:37,2020-03-25,08:15:37.000000


In [ ]:
import time
#time.mktime(mmsi_205792000['epochMillis_new'].timetuple())
#df.a.dt.microsecond
mmsi_205792000['diff'] = (mmsi_205792000.iloc[:,27].diff())
#to ms
mmsi_205792000['diff'] = mmsi_205792000['diff'].apply(lambda x: x.total_seconds() * 1000.0)

In [ ]:
#mmsi_205792000_time = (mmsi_205792000['epochMillis'].sort_values().tail(1).values[0])-(mmsi_205792000['epochMillis'].sort_values().head(1).values[0])
#print("lead time (in Milliseconds): ", (mmsi_205792000_time/10**6))

## Answer:

In [ ]:
mmsi_205792000.loc[:,['mmsi','navigation.navCode', 'navigation.navDesc','date', 'time', 'diff']]

,mmsi,navigation.navCode,navigation.navDesc,date,time,diff
56121,205792000,1,At Anchor,2020-03-23,20:13:31.000000,NaN
61738,205792000,5,Moored,2020-03-24,13:26:20.000000,61969000.0
61748,205792000,5,Moored,2020-03-24,13:50:20.000000,1440000.0
68059,205792000,1,At Anchor,2020-03-23,11:13:30.000000,-95810000.0
83560,205792000,5,Moored,2020-03-25,01:38:20.000000,138290000.0
...,...,...,...,...,...,...
3366678,205792000,1,At Anchor,2020-03-23,17:37:31.000000,-25499000.0
3372967,205792000,1,At Anchor,2020-03-23,13:13:31.000000,-15840000.0
3399332,205792000,0,Under Way Using Engine,2020-03-24,03:03:10.000000,49779000.0
3424055,205792000,0,Under Way Using Engine,2020-03-25,08:15:37.000000,105147000.0


# 8 For MMSI = 413970021, provide the same report as number 7
<ol>Do you agree with the Navigation Code(s) and Description(s) for this particular vessel?</ol>
<li>If you do agree, provide an explanation why you agree.</li>
<li>If you do not agree, provide an explanation why do disagree.</li>
<li>Additionally, if you do not agree, what would you change it to and why? </li>

In [ ]:
mmsi_413970021 = df.loc[ (df['mmsi'] == 413970021) &
       ((df['navigation.navDesc'] == navigation.index[0]) | 
       ( df['navigation.navDesc'] == navigation.index[1]) |  
       ( df['navigation.navDesc'] == navigation.index[2]) | 
       ( df['navigation.navDesc'] == navigation.index[3]) |  
       ( df['navigation.navDesc'] == navigation.index[4])), :].copy()

#df.loc[(df['mmsi'] == 413970021) , : ]['navigation.navDesc'].value_counts() # 'Unknown'  ['navigation.navDesc'] in top-5
mmsi_413970021

,epochMillis,mmsi,olson_timezone,imo,callSign,destination,position.latitude,position.longitude,navigation.navCode,navigation.navDesc,navigation.courseOverGround,navigation.heading,navigation.rateOfTurn,navigation.speedOverGround,vesselDetails.name,vesselDetails.typeName,vesselDetails.typeCode,vesselDetails.draught,vesselDetails.length,vesselDetails.width,vesselDetails.flagCode,vesselDetails.flagCountry,port.unlocode,port.name,port.latitude,port.longitude,cargoDetails,epochMillis_new,date,time
36514,1553326724000,413970021,Asia/Shanghai,2112.0,TXYR0,000,31.326313,121.554600,16,Unknown,273.4,0.0,0.0,0.0,XIANG HAO 08,Cargo,70,0.0,86.0,15.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,0,2019-03-23 07:38:44,2019-03-23,07:38:44.000000
37019,1553300534000,413970021,Asia/Shanghai,2112.0,TXYR0,000,31.326345,121.554540,16,Unknown,108.3,0.0,0.0,0.0,XIANG HAO 08,Cargo,70,0.0,86.0,15.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,0,2019-03-23 00:22:14,2019-03-23,00:22:14.000000
37021,1553302339000,413970021,Asia/Shanghai,2112.0,TXYR0,000,31.326290,121.554565,16,Unknown,61.1,0.0,0.0,0.0,XIANG HAO 08,Cargo,70,0.0,86.0,15.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,0,2019-03-23 00:52:19,2019-03-23,00:52:19.000000
37044,1553340485000,413970021,Asia/Shanghai,2112.0,TXYR0,000,31.326326,121.554570,16,Unknown,158.5,0.0,0.0,0.0,XIANG HAO 08,Cargo,70,0.0,86.0,15.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,0,2019-03-23 11:28:05,2019-03-23,11:28:05.000000
37376,1553309779000,413970021,Asia/Shanghai,2112.0,TXYR0,000,31.326351,121.554590,16,Unknown,180.1,0.0,0.0,0.0,XIANG HAO 08,Cargo,70,0.0,86.0,15.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,0,2019-03-23 02:56:19,2019-03-23,02:56:19.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3340291,1553810430000,413970021,Asia/Shanghai,2112.0,TXYR0,000,31.682968,121.194640,16,Unknown,300.8,0.0,0.0,0.0,XIANG HAO 08,Cargo,70,0.0,86.0,15.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,0,2019-03-28 22:00:30,2019-03-28,22:00:30.000000
3340714,1553616200000,413970021,Asia/Shanghai,2112.0,TXYR0,000,31.682373,121.197080,16,Unknown,193.7,0.0,0.0,0.0,XIANG HAO 08,Cargo,70,0.0,86.0,15.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,0,2019-03-26 16:03:20,2019-03-26,16:03:20.000000
3340843,1553549765000,413970021,Asia/Shanghai,2112.0,TXYR0,000,31.546364,121.343430,16,Unknown,302.6,0.0,0.0,7.2,XIANG HAO 08,Cargo,70,0.0,86.0,15.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,0,2019-03-25 21:36:05,2019-03-25,21:36:05.000000
3341492,1553712746000,413970021,Asia/Shanghai,2112.0,TXYR0,000,31.682281,121.197030,16,Unknown,121.3,0.0,0.0,0.0,XIANG HAO 08,Cargo,70,0.0,86.0,15.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,0,2019-03-27 18:52:26,2019-03-27,18:52:26.000000


In [ ]:
#time.mktime(mmsi_413970021['epochMillis_new'].timetuple())

mmsi_413970021['diff'] = (mmsi_413970021.iloc[:,27].diff())
#to ms
mmsi_413970021['diff'] = mmsi_413970021['diff'].apply(lambda x: x.total_seconds() * 1000.0)

# Answer:
I agree, this ia a particular vessel (Cargo	'XIANG HAO 08').

In [ ]:
mmsi_413970021.loc[:,['mmsi','navigation.navCode', 'navigation.navDesc','date', 'time', 'diff']]

,mmsi,navigation.navCode,navigation.navDesc,date,time,diff
36514,413970021,16,Unknown,2019-03-23,07:38:44.000000,NaN
37019,413970021,16,Unknown,2019-03-23,00:22:14.000000,-26190000.0
37021,413970021,16,Unknown,2019-03-23,00:52:19.000000,1805000.0
37044,413970021,16,Unknown,2019-03-23,11:28:05.000000,38146000.0
37376,413970021,16,Unknown,2019-03-23,02:56:19.000000,-30706000.0
...,...,...,...,...,...,...
3340291,413970021,16,Unknown,2019-03-28,22:00:30.000000,438604000.0
3340714,413970021,16,Unknown,2019-03-26,16:03:20.000000,-194230000.0
3340843,413970021,16,Unknown,2019-03-25,21:36:05.000000,-66435000.0
3341492,413970021,16,Unknown,2019-03-27,18:52:26.000000,162981000.0


In [ ]:
mmsi_413970021.pivot_table(['vesselDetails.name'], ['vesselDetails.typeName'], aggfunc=(np.max)).sort_values(['vesselDetails.name'])

,vesselDetails.name
vesselDetails.typeName,
Cargo,XIANG HAO 08


# 9 For each of the time period(s) from item three, provide a tabulation of the top 10 series of vessel navigation code/description ordered states.


In [ ]:
df

,epochMillis,mmsi,olson_timezone,imo,callSign,destination,position.latitude,position.longitude,navigation.navCode,navigation.navDesc,navigation.courseOverGround,navigation.heading,navigation.rateOfTurn,navigation.speedOverGround,vesselDetails.name,vesselDetails.typeName,vesselDetails.typeCode,vesselDetails.draught,vesselDetails.length,vesselDetails.width,vesselDetails.flagCode,vesselDetails.flagCountry,port.unlocode,port.name,port.latitude,port.longitude,cargoDetails,epochMillis_new,date,time
0,1553847836000,413372730,Asia/Shanghai,0.0,FYUA$,SHANGHAI,31.334450,121.550385,16,Unknown,317.3,0.0,0.0,0.0,TONGYUAN10,Tanker,80,0.3,42.0,9.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-29 08:23:56,2019-03-29,08:23:56
1,1553848022000,412018031,Asia/Shanghai,NaN,00371,NaN,30.844887,122.295370,16,Unknown,128.7,480.0,0.0,8.2,ZHEPINHUYU00371,Fishing,30,NaN,28.0,4.0,412,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-29 08:27:02,2019-03-29,08:27:02
2,1553848135000,413444250,Asia/Shanghai,0.0,BLGS8,DONG YING,30.723333,122.274870,0,Under Way Using Engine,26.5,28.0,0.0,11.3,XING LONG ZHOU 172,Tanker,80,4.5,119.0,18.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-29 08:28:55,2019-03-29,08:28:55
3,1553848288000,412378760,Asia/Shanghai,2176.0,BIAL2,YUEQINWAN,31.333086,121.557630,16,Unknown,0.0,0.0,0.0,0.0,ZHONGYUANRANGONG 01,Tanker,83,9.2,53.0,9.0,412,China,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2019-03-29 08:31:28,2019-03-29,08:31:28
4,1553848430000,412365857,Asia/Shanghai,NaN,NaN,NaN,32.118640,121.499466,16,Unknown,319.0,0.0,0.0,0.2,SUHAIMENYU16271,Unknown,100,NaN,NaN,NaN,412,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-29 08:33:50,2019-03-29,08:33:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3473872,1553471408000,413764058,Asia/Shanghai,0.0,NaN,NaN,30.947575,121.216095,16,Unknown,2.0,0.0,0.0,7.8,HU HUAN HUO 30,Other,90,0.0,17.0,5.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-24 23:50:08,2019-03-24,23:50:08
3473873,1553471514000,566400000,Asia/Shanghai,9736638.0,9V3031,SHANGAI,31.090883,121.463104,0,Under Way Using Engine,102.4,158.0,0.0,0.1,MAREX NOA,Tanker,84,7.0,124.0,20.0,566,Singapore,CNSHG,SHANGHAI PT,31.334994,121.659069,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",2019-03-24 23:51:54,2019-03-24,23:51:54
3473874,1553471653000,412760770,Asia/Shanghai,0.0,BLGL7,GUANG ZHOU,30.746483,122.932560,0,Under Way Using Engine,181.5,183.0,0.0,10.0,HONG PU 96,Cargo,79,8.0,140.0,20.0,412,China,CNSHG,SHANGHAI PT,31.334994,121.659069,No Additional Information,2019-03-24 23:54:13,2019-03-24,23:54:13
3473875,1553471750000,413373640,Asia/Shanghai,0.0,NaN,SHANGHAI,31.391459,121.662190,5,Moored,26.6,0.0,0.0,0.0,ZHEN HUA TUO 12,Tug,52,3.5,0.0,0.0,413,China,CNSHG,SHANGHAI PT,31.334994,121.659069,NaN,2019-03-24 23:55:50,2019-03-24,23:55:50


In [ ]:
df['year'] = df['date'].apply(lambda x: x.strftime('%Y'))

In [ ]:
period_1 = df.loc[(df['year'] == 2019),:].copy()
period_2 = df.loc[(df['year'] == 2020),:].copy()

In [ ]:
#pd.to_datetime(2020, unit='year')
pd.to_datetime('2020', format='%Y')


Timestamp('2020-01-01 00:00:00')

# Answer:

In [ ]:
print('period 1 (2019):')
period_2019_top10 = (pd.crosstab(period_1['navigation.navCode'], period_1['navigation.navDesc']).sum()).sort_values(ascending=False)[:10]
period_2019_top10

period 1 (2019):


navigation.navDesc
Unknown                       774769
Under Way Using Engine        536139
Moored                        261009
At Anchor                     176543
Not Defined                    16272
Underway Sailing               11530
Restricted Manoeuvrability      4466
Reserved For Future Use         2159
Not Under Command               1613
Constrained By Her Draught      1483
dtype: int64

In [ ]:
print('period 2 (2020):')
period_2020_top10 = pd.crosstab(period_2['navigation.navCode'], period_2['navigation.navDesc']).sum().sort_values(ascending=False)[:10]
period_2020_top10 

period 2 (2020):


navigation.navDesc
Unknown                       583216
Under Way Using Engine        527537
Moored                        293124
At Anchor                     249890
Underway Sailing               13359
Not Defined                    13058
Restricted Manoeuvrability      3771
Not Under Command               1858
Engaged In Fishing               131
Aground                          104
dtype: int64

# 10 Using the results from item 9, compare the volume of each vessel navigation code/description ordered states for each time period(s) from item three. 
<li>Which increased/decreased?</li>

In [ ]:
#period_2020_top10.describe()# 
for i in period_2019_top10.index:
    if i not in period_2020_top10.index:
        print(i, ' - decreased, not in top10 now')
    elif  period_2019_top10[i] < period_2020_top10[i]:
        print(i,' - increased')      
    else:
        print(i, ' - decreased')

Unknown  - decreased
Under Way Using Engine  - decreased
Moored  - increased
At Anchor  - increased
Not Defined  - decreased
Underway Sailing  - increased
Restricted Manoeuvrability  - decreased
Reserved For Future Use  - decreased, not in top10 now
Not Under Command  - increased
Constrained By Her Draught  - decreased, not in top10 now


# 11 For each of the time period(s) from item three and using only the “At Anchor” and “Moored” navigation descriptions, quantify the average “dwell”

# Answer:

In [ ]:
%%time
# period 1:

period_1[(period_1['navigation.navDesc'] == 'At Anchor') | (period_1['navigation.navDesc'] == 'Moored')]['epochMillis_new'].diff()

CPU times: user 1.3 s, sys: 8.96 ms, total: 1.31 s
Wall time: 1.31 s


5                       NaT
7           0 days 00:05:50
9           0 days 00:04:33
25          0 days 00:40:48
27          0 days 00:04:21
                 ...       
3473850   -1 days +23:57:04
3473864     0 days 00:02:25
3473870   -1 days +23:56:32
3473875     0 days 00:09:34
3473876     0 days 00:02:26
Name: epochMillis_new, Length: 437552, dtype: timedelta64[ns]

In [ ]:
#
period_2[(period_2['navigation.navDesc'] == 'At Anchor') | (period_2['navigation.navDesc'] == 'Moored')]['epochMillis_new'].diff()


847                     NaT
849         0 days 00:03:52
853         0 days 00:09:48
855         0 days 00:03:48
857         0 days 00:04:23
                 ...       
3473781     0 days 12:30:23
3473856   -1 days +11:17:07
3473857     0 days 00:07:12
3473858     0 days 00:09:16
3473860     0 days 00:16:42
Name: epochMillis_new, Length: 543014, dtype: timedelta64[ns]

# 12 Describe or show how you would quantify if the difference(s) in “dwells” between the time-period(s) is(are) significant.

In [ ]:
#period_1[(period_1['navigation.navDesc'] == 'At Anchor') | (period_1['navigation.navDesc'] == 'Moored')]['epochMillis_new'].diff()

dwells = ((data_time[data_time['date'] > pd.to_datetime('2020',
             format='%Y')]['date']).sort_values(ascending=False).tail(1).values) - ((data_time[data_time['date'] < pd.to_datetime('2020',  
             format='%Y')]['date']).sort_values(ascending=True).tail(1).values)

In [ ]:
print( "Difference between the time-period(s) is", dwells[0])

Difference between the time-period(s) is 30844800000000000 nanoseconds


# 13 Describe or show how you would create a Machine Learning Model to predict “dwell” times for the region.
Bonus Points: Provide the code and performance results for your machine learning model on an OOB sample.
In respect of time, performance of the ML model is not important and the number of features can be minimal. What mattes is
that the code works and you can explain your thought process if asked.

#Answer: https://www.researchgate.net/publication/304530252_Development_of_Models_Predicting_Dwell_Time_of_Import_Containers_in_Port_Container_Terminals_-_An_Artificial_Neural_Networks_Application